### Importing

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/new_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['Unnamed: 0', 'id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
X.sample(5)

In [ ]:
X.columns

### Preprocessing

In [40]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [41]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od']),
    ('num', num_pipeline, ['date_difference', 'trip_type', 'branded_fare', 'number_of_pax', 'ADVS_price', 'PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory'])
  ], remainder='drop')

In [42]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

In [ ]:
X_train_prep[0]

### Training

In [ ]:
from xgboost import XGBClassifier

bst = XGBClassifier(learning_rate=.3, objective='multi:softmax')
bst.fit(X_train_prep, y_train)


y_pred = bst.predict(X_test_prep)
accuracy_score(y_pred, y_test)

### Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],           # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.2],         # Step size shrinkage
    'max_depth': [3, 5, 7],                    # Maximum depth of the trees
    'min_child_weight': [1, 3, 5],             # Minimum sum of instance weight needed in a child
    'subsample': [0.6, 0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0],       # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction to make a further partition on a leaf node
}

rnd_search = GridSearchCV(bst,
    param_grid=param_grid, verbose=3, cv=3, n_jobs=-1,
    scoring='neg_root_mean_squared_error')

rnd_search.fit(X_train_prep, y_train)

### Predicting

In [ ]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = bst.predict(X_to_predict_prep)

In [ ]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

In [25]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [27]:
pd.DataFrame.to_csv(y_final, '../predictions/pred2.csv', index=False)